In [2]:
import socket

def send_public_paramters_to_Alice(parameter):
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_address = ('localhost', 12345)
    server_socket.bind(server_address)
    server_socket.listen(1)

    print('Waiting for a connection...')

    client_socket, client_address = server_socket.accept()
    print('Connected by', client_address)

    # Send an integer to the client
    
    client_socket.sendall(str(parameter).encode('utf-8'))
    client_socket.close()
    server_socket.close()
def read_Aliced_public_key():
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_address = ('localhost', 1111)
    client_socket.connect(server_address)

    # Receive data from the server
    data = client_socket.recv(1024)
    received_integer = int(data.decode('utf-8'))
    print('Received integer:', received_integer)
    client_socket.close()
    return received_integer

    


In [5]:
from sympy import randprime

def count_digits(number):
    # Convert the number to a string and calculate its length
    num_str = str(number)
    num_digits = len(num_str)
    return num_digits

# Choose a large prime 'p'

p = randprime(2**1024, 2**2048)

from random import randint
g=randint(2,p-2)

bin_p=bin(p)
count_digits(bin_p)

2050

In [6]:
import random
#Miller-Rabin test
def is_probably_prime(n, k):
    if n <= 1:
        return False
    if n == 2 or n == 3:
        return True
    if n % 2 == 0:
        return False

    # Write n as 2^r * d + 1
    r, d = 0, n - 1
    while d % 2 == 0:
        r += 1
        d //= 2

    # Witness loop
    for _ in range(k):
        a = random.randint(2, n - 2)
        x = pow(a, d, n)

        if x == 1 or x == n - 1:
            continue

        for _ in range(r - 1):
            x = pow(x, 2, n)
            if x == n - 1:
                break
        else:
            return False

    return True
rounds_of_testing = 5
if is_probably_prime(p, rounds_of_testing):
  print(" is probably a prime number.")
else:
  print(" is composite.")

 is probably a prime number.


In [7]:
send_public_paramters_to_Alice(p)

Waiting for a connection...
Connected by ('127.0.0.1', 55175)


In [9]:
from random import randint
g=randint(2,p-2)


In [11]:
send_public_paramters_to_Alice(g)

Waiting for a connection...
Connected by ('127.0.0.1', 55229)


In [12]:

Bob_private_key = randint(2, p - 2)
print("Bob_Private_key:\n",Bob_private_key)

Bob_Private_key:
 8460805340771535096058292169395124757676517656266851978828809539351014095113222904824844508886686710616611425243387915966023988787028129475880296167267918680388962597173620179091926184276506618475033817377149773498314405646819947088830866188665766867989462326155917521995574626145375250619590342225395071393860878762626603630358909962385101352510461797556348661223148043935874620853706379024516154988120510840704203444832653627409634519365382452451540305246532811022755872565344708420966764036450703131282512639410580599714606282653662889935052265046249038356889099608121765953872234994084548857389230995948580383732


In [13]:
def mod_exp(base, exponent, modulus):
    result = 1
    base = base % modulus
    while exponent > 0:
        if exponent % 2 == 1:
            result = (result * base) % modulus
        exponent //= 2
        base = (base * base) % modulus
    return result

In [14]:
Bob_public_key = mod_exp(g, Bob_private_key, p)
print("Bob_Public_key:\n",Bob_public_key)

Bob_Public_key:
 2307318120642966431663895199235877783225365729269979662677006727111535525222854121367449550341174836187579056507538792328714007508239526482436191494275110813615815332583991811063098202105034970522270453920300456717388297352062295479014502586860736353310756785886272344052878535340277689769200867391101408830843600993647472708168576212987824871926232147215901213493643187656388330901838317333956730119421081956324006695445649829440322069918931866372561238031833759876524703955313813586775554922568795895346391982007127814923848568476865091698260592587113306538534118837637829286592397006111814798056892865727531998554


In [15]:
send_public_paramters_to_Alice(Bob_public_key)

Waiting for a connection...
Connected by ('127.0.0.1', 55272)


In [16]:
Alice_public_key=read_Aliced_public_key()

Received integer: 7129299296205506582106390138532099103738059394779090673931635624853055737933529134852343042931110849691578081400772307541188106543861531361066057363268964163961856907201609159852933802887939546796275142955655800291999368957889672284327496300130049997876429440264766799441229944880255481186975048028909229179335586394091809985414120596359397309200738127576385968398536482653836193017462716150281326093735025175430195687561737333567576282882101496751497737634411055528874652251007376530751136774677645878617342927927064613816639900284373320715455316596220056595767914947397801425951483282714096277225778959887320682132


In [18]:
Bob_common_key = mod_exp(Alice_public_key, Bob_private_key, p)
print("The common key calculated by Bob:\n",Bob_common_key)

The common key calculated by Bob:
 5272430107689889097796882842610122745648554468770498663219568113872026188333003156976891029175501913106236541955306563766350079827396832650628506723788137581421519737847006894122314462787409524471745092924827744272669737221005493460648647127975978015324848224050557401217996594479640782598114631839962397856336983935782017444628415724269679739912528520160499286677668549067205662483072170790787336480267053492579547329706096943112809192329777259952470722285790890709063335185949459694744488691851225665518326894656357685888721847145326591059889122456731226181735870141684045998786988139018748063668157906261608528929


In [19]:
#convert common key to bytes
Bob_common_key_bytes=Bob_common_key.to_bytes(10000, byteorder='big')

In [20]:
import hashlib

def derive_key(original_key, output_length=16):
    # Use SHA-256 hash function
    sha256 = hashlib.sha256()

    # Update the hash object with the original key
    sha256.update(original_key)

    # Get the digest (hash) and truncate it to the desired length
    derived_key = sha256.digest()[:output_length]

    return derived_key

In [21]:
#get 16 bytes from common key using hash function

Bob_derive_key=derive_key(Bob_common_key_bytes)
Bob_derive_key

b'\x04v5\xc1~G\x16B\x89\x90E`p\xf6\xa0v'

In [23]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding
from pydub import AudioSegment
import os

# Set the path to the ffmpeg executable
AudioSegment.converter = "ffmpeg.exe"

def decrypt_audio(input_audio_path, output_audio_path, key):
    # Load the encrypted audio file
    encrypted_audio = AudioSegment.from_file(input_audio_path)

    # Convert audio to raw data (bytearray)
    encrypted_data = encrypted_audio.raw_data

    # Extract IV from the encrypted data
    iv = encrypted_data[:16]
    encrypted_data = encrypted_data[16:]

    # Create an AES cipher object with the key, mode, and IV
    cipher = Cipher(algorithms.AES(key), modes.CFB(iv), backend=default_backend())
    decryptor = cipher.decryptor()

    # Decrypt the data
    decrypted_data = decryptor.update(encrypted_data) + decryptor.finalize()

    # Unpad the data
    unpadder = padding.PKCS7(algorithms.AES.block_size).unpadder()
    audio_data = unpadder.update(decrypted_data) + unpadder.finalize()

    # Save the decrypted audio to a new file
    decrypted_audio = AudioSegment(audio_data, frame_rate=encrypted_audio.frame_rate, sample_width=encrypted_audio.sample_width, channels=encrypted_audio.channels)
    decrypted_audio.export(output_audio_path, format="wav")

In [24]:
encrypted_audio_path = 'encrypted_audio.wav'
decrypted_audio_path = 'decrypted_audio.wav'

decrypt_audio(encrypted_audio_path, decrypted_audio_path, Bob_derive_key)

In [25]:
import pygame

def play_audio(file_path):
    pygame.init()
    pygame.mixer.init()

    try:
        # Load the audio file
        pygame.mixer.music.load(file_path)

        # Play the audio file
        pygame.mixer.music.play()

        # Wait for the audio to finish playing
        while pygame.mixer.music.get_busy():
            pygame.time.Clock().tick(10)

    except pygame.error as e:
        print(f"Error playing audio: {e}")
    finally:
        # Cleanup
        pygame.mixer.quit()
        pygame.quit()



pygame 2.5.2 (SDL 2.28.3, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [26]:
play_audio("decrypted_audio.wav")